<a href="https://colab.research.google.com/github/ainsley-snell/Data_Mining_CS290/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import torch
import tiktoken

In [4]:
!wget https://raw.githubusercontent.com/ainsley-snell/Data_Mining_CS290/main/SILVERBLAZE.txt

with(open("SILVERBLAZE.txt", "r", encoding="utf-8")) as f:
    raw_text = f.read()
raw_text[:50]

--2025-12-13 19:10:45--  https://raw.githubusercontent.com/ainsley-snell/Data_Mining_CS290/main/SILVERBLAZE.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52531 (51K) [text/plain]
Saving to: ‘SILVERBLAZE.txt.2’

SILVERBLAZE.txt.2   100%[===================>]  51.30K  --.-KB/s    in 0.005s  

2025-12-13 19:10:45 (10.6 MB/s) - ‘SILVERBLAZE.txt.2’ saved [52531/52531]



'SILVER BLAZE\n\n"I AM afraid, Watson, that I shall h'

In [5]:
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
enc_text = tokenizer.encode(raw_text)

In [7]:
print(enc_text[:20])

[50, 4146, 5959, 9878, 32, 21211, 198, 198, 1, 40, 3001, 7787, 11, 14959, 11, 326, 314, 2236, 423, 284]


In [8]:
print( tokenizer.decode( enc_text[:2]))

SIL


In [9]:
len( enc_text)

12491

In [10]:
for i in range(1,10):
    print("Input:", tokenizer.decode(enc_text[:i]), "Target:", tokenizer.decode([enc_text[i]]))

Input: S Target: IL
Input: SIL Target: VER
Input: SILVER Target:  BL
Input: SILVER BL Target: A
Input: SILVER BLA Target: ZE
Input: SILVER BLAZE Target: 

Input: SILVER BLAZE
 Target: 

Input: SILVER BLAZE

 Target: "
Input: SILVER BLAZE

" Target: I


In [11]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [12]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [13]:

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   50,  4146,  5959,  9878],
        [   32, 21211,   198,   198],
        [    1,    40,  3001,  7787],
        [   11, 14959,    11,   326],
        [  314,  2236,   423,   284],
        [  467,   553,   531, 17628],
        [   11,   355,   356,  3332],
        [  866,  1978,   284,   674]])

Targets:
 tensor([[ 4146,  5959,  9878,    32],
        [21211,   198,   198,     1],
        [   40,  3001,  7787,    11],
        [14959,    11,   326,   314],
        [ 2236,   423,   284,   467],
        [  553,   531, 17628,    11],
        [  355,   356,  3332,   866],
        [ 1978,   284,   674, 12607]])


In [14]:
# to apply the tokenizer's decoder to these IDs, the rows of the tensor `inputs` have to be converted into lists:
for row in inputs:
    print( tokenizer.decode( row.tolist() ) )

SILVER BL
AZE


"I AM afraid
, Watson, that
 I shall have to
 go," said Holmes
, as we sat
 down together to our


In [15]:

# we don't send these IDs to the LLM for training; we associate a vector a.k.a. tensor with each ID and then train the LLM on the vectors
# as a first example, let's create embedding vectors of length 3 for each token in a vocabulary of 6 tokens
vocab_size = 6
output_dim = 3
embedding = torch.nn.Embedding( vocab_size, output_dim )
print(embedding.weight)

Parameter containing:
tensor([[ 0.7076, -0.4452, -0.5832],
        [ 0.8526,  1.1339,  1.5151],
        [ 1.4928, -0.8930, -1.5079],
        [ 0.7168,  0.1400,  0.0474],
        [ 0.0139, -0.9869,  1.1930],
        [ 2.5283, -0.9062, -0.1422]], requires_grad=True)


In [16]:

# if you just want the tensor part of this without the requires_grad=True bit
# method 1:
embedding.weight.data

tensor([[ 0.7076, -0.4452, -0.5832],
        [ 0.8526,  1.1339,  1.5151],
        [ 1.4928, -0.8930, -1.5079],
        [ 0.7168,  0.1400,  0.0474],
        [ 0.0139, -0.9869,  1.1930],
        [ 2.5283, -0.9062, -0.1422]])

In [17]:
# if you just want the tensor part of this without the requires_grad=True bit
# method 1:
embedding.weight.detach()

tensor([[ 0.7076, -0.4452, -0.5832],
        [ 0.8526,  1.1339,  1.5151],
        [ 1.4928, -0.8930, -1.5079],
        [ 0.7168,  0.1400,  0.0474],
        [ 0.0139, -0.9869,  1.1930],
        [ 2.5283, -0.9062, -0.1422]])

In [19]:
# call this A for some examples:
A = embedding.weight.detach()

In [20]:
# first row:
A[0]

tensor([ 0.7076, -0.4452, -0.5832])

In [21]:
# second row:
A[1]

tensor([0.8526, 1.1339, 1.5151])

In [22]:
# first column:
A[:,0]

tensor([0.7076, 0.8526, 1.4928, 0.7168, 0.0139, 2.5283])

In [23]:
# element in row 2, column 3:
A[1,2]

tensor(1.5151)

In [24]:
# to create a tensor directly:
x = torch.tensor([1.2,2.1])
y = torch.tensor([2.7,1.5])
print(x)
print(y)

tensor([1.2000, 2.1000])
tensor([2.7000, 1.5000])


In [25]:
torch.dot( x,y)

tensor(6.3900)

In [26]:
# check:
1.2*2.7 + 2.1*1.5

6.390000000000001